In [28]:
!pip install folium
!pip install geocoder

     |████████████████████████████████| 102kB 13.5MB/s ta 0:00:01


In [29]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import lxml.html as lh
import geocoder

In [30]:
#scraping data

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [31]:
tr_elements = doc.xpath('//tr')

col=[]
i=0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))
    
col

[('Postcode', []), ('Borough', []), ('Neighbourhood\n', [])]

In [32]:
# T =rows
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    if len(T)!=3:
        break
    i=0
    
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [33]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df = df.replace('\n','', regex=True)

In [34]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df = df.reset_index(drop=True)
df = df.rename(columns={'Neighbourhood\n': 'Neighbourhood'})
df["Neighbourhood"] = df["Neighbourhood"].str.strip()

In [35]:

df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [36]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighbourhood
5,M7A,Queen's Park,Queen's Park


In [37]:
df.shape

(210, 3)

In [38]:
import geocoder

link = "http://cocl.us/Geospatial_data"
df1 = pd.read_csv(link)

df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [39]:
df1.shape

(103, 3)

In [40]:
df1.columns = ['Postcode','Latitude','Longitude']

cols = df1.columns.tolist()
cols

['Postcode', 'Latitude', 'Longitude']

In [41]:
neighborhoods = pd.merge(df, df1, on='Postcode')
neighborhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


In [43]:
print('The dataframe consists of {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]))


The dataframe consists of 11 boroughs and 210 neighborhoods.


In [44]:
#starting point  to create the map 
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [45]:
# we are now creating a map using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers 
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [46]:
# getting info from foursquare as in the lab
LIMIT = 100

CLIENT_ID = 'BS02VEGYA4RGGSKPNNIHWSDL1YWFSNF5TESJXJZ40YTWZ2PV' 
CLIENT_SECRET = 'SUFLRGVPIEFIBDKAOF0ZGED04BZFKW0G0IWRZXWSZCRFEI44' 
VERSION = '20180604'

In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        results = requests.get(url).json()["response"]['groups'][0]['items']
        
       
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'], latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'])

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Queen's Park
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design Exchange
Toronto 

In [49]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [50]:
toronto_venues.shape

(4306, 7)

In [51]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
Agincourt North,2,2,2,2,2,2
Albion Gardens,8,8,8,8,8,8
Alderwood,10,10,10,10,10,10
Bathurst Manor,19,19,19,19,19,19
Bathurst Quay,16,16,16,16,16,16
Bayview Village,4,4,4,4,4,4
Beaumond Heights,8,8,8,8,8,8


In [53]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 277 unique categories.


**Analyze each neighborhood

In [54]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
toronto_onehot.head()

(4306, 277)


,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,

In [55]:

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.0,

In [56]:
toronto_grouped.shape

(199, 277)

In [57]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2        Steakhouse  0.04
3               Bar  0.04
4  Asian Restaurant  0.03


----Agincourt----
                       venue  freq
0                     Lounge   0.2
1             Clothing Store   0.2
2               Skating Rink   0.2
3  Latin American Restaurant   0.2
4             Breakfast Spot   0.2


----Agincourt North----
                      venue  freq
0                      Park   0.5
1                Playground   0.5
2  Mediterranean Restaurant   0.0
3         Mobile Phone Shop   0.0
4        Miscellaneous Shop   0.0


----Albion Gardens----
                  venue  freq
0         Grocery Store  0.25
1  Fast Food Restaurant  0.12
2              Pharmacy  0.12
3           Pizza Place  0.12
4   Fried Chicken Joint  0.12


----Alderwood----
          venue  freq
0   Pizza Place   0.2
1  Dance Studio   0.1
2  Skating Rink   0.1
3           Gym   0.1
4      Pharmacy   0.1


----Bath

                  venue  freq
0  Gym / Fitness Center  0.25
1                   Gym  0.25
2         Grocery Store  0.25
3        Discount Store  0.25
4           Yoga Studio  0.00


----Downsview West----
           venue  freq
0           Park   0.2
1          Hotel   0.2
2           Bank   0.2
3  Grocery Store   0.2
4  Shopping Mall   0.2


----Dufferin----
         venue  freq
0       Bakery  0.12
1     Pharmacy  0.12
2  Supermarket  0.12
3         Café  0.06
4          Bar  0.06


----East Birchmount Park----
               venue  freq
0     Discount Store  0.29
1        Coffee Shop  0.14
2         Hobby Shop  0.14
3  Convenience Store  0.14
4   Department Store  0.14


----East Toronto----
                 venue  freq
0                 Park   0.4
1  Rental Car Location   0.2
2          Coffee Shop   0.2
3    Convenience Store   0.2
4     Malay Restaurant   0.0


----Emery----
                    venue  freq
0  Furniture / Home Store   0.5
1          Baseball Field   0.5
2         

                        venue  freq
0  Construction & Landscaping  0.25
1              Baseball Field  0.25
2               Deli / Bodega  0.25
3            Business Service  0.25
4         Monument / Landmark  0.00


----Kingsway Park South West----
                  venue  freq
0        Hardware Store  0.08
1                   Gym  0.08
2         Burrito Place  0.08
3  Fast Food Restaurant  0.08
4        Sandwich Place  0.08


----L'Amoreaux East----
                      venue  freq
0                      Park   0.5
1                Playground   0.5
2  Mediterranean Restaurant   0.0
3         Mobile Phone Shop   0.0
4        Miscellaneous Shop   0.0


----L'Amoreaux West----
                  venue  freq
0  Fast Food Restaurant  0.15
1    Chinese Restaurant  0.15
2           Pizza Place  0.08
3         Grocery Store  0.08
4       Bubble Tea Shop  0.08


----Lawrence Heights----
                    venue  freq
0          Clothing Store   0.2
1  Furniture / Home Store   0.1
2       Ac

              venue  freq
0  Airport Terminal  0.12
1    Airport Lounge  0.12
2   Airport Service  0.12
3     Boat or Ferry  0.06
4  Sculpture Garden  0.06


----Rathnelly----
                venue  freq
0         Coffee Shop  0.13
1                 Pub  0.13
2          Restaurant  0.07
3        Liquor Store  0.07
4  Light Rail Station  0.07


----Richmond----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2        Steakhouse  0.04
3               Bar  0.04
4  Asian Restaurant  0.03


----Richview Gardens----
               venue  freq
0               Park  0.25
1        Pizza Place  0.25
2  Mobile Phone Shop  0.25
3           Bus Line  0.25
4        Yoga Studio  0.00


----Riverdale----
                    venue  freq
0        Greek Restaurant  0.22
1             Coffee Shop  0.10
2          Ice Cream Shop  0.07
3  Furniture / Home Store  0.05
4              Restaurant  0.05


----Roncesvalles----
                venue  freq
0           Gift Shop  0.15
1

                  venue  freq
0        Hardware Store  0.08
1                   Gym  0.08
2         Burrito Place  0.08
3  Fast Food Restaurant  0.08
4        Sandwich Place  0.08


----Thistletown----
                  venue  freq
0         Grocery Store  0.25
1  Fast Food Restaurant  0.12
2              Pharmacy  0.12
3           Pizza Place  0.12
4   Fried Chicken Joint  0.12


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.11
1       Burger Joint  0.11
2        Yoga Studio  0.05
3               Bank  0.05
4                Gym  0.05


----Toronto Dominion Centre----
         venue  freq
0  Coffee Shop  0.12
1         Café  0.08
2        Hotel  0.05
3   Restaurant  0.05
4       Bakery  0.04


----Toronto Islands----
            venue  freq
0     Coffee Shop  0.13
1        Aquarium  0.05
2            Café  0.04
3           Hotel  0.04
4  Scenic Lookout  0.03


----Trinity----
                   venue  freq
0                    Bar  0.13
1            Coffee

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
#top 10 venues for each neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Bar,Steakhouse,Sushi Restaurant,Thai Restaurant,Salad Place,Restaurant,Burger Joint,Bakery
1,Agincourt,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Clothing Store,Women's Store,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore
2,Agincourt North,Park,Playground,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
3,Albion Gardens,Grocery Store,Fast Food Restaurant,Pharmacy,Sandwich Place,Fried Chicken Joint,Pizza Place,Beer Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant
4,Alderwood,Pizza Place,Skating Rink,Sandwich Place,Dance Studio,Gym,Coffee Shop,Pub,Pharmacy,Pool,Diner


In [65]:
#cluster neighborhood

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [68]:
toronto_merged.head()# cluster labels are inserted.

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Construction & Landscaping,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Intersection,Financial or Legal Service,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1.0,Coffee Shop,Bakery,Park,Pub,Restaurant,Café,Mexican Restaurant,Breakfast Spot,Ice Cream Shop,Bank
3,M6A,North York,Lawrence Heights,43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Gift Shop,Sporting Goods Shop,Miscellaneous Shop,Coffee Shop,Boutique,Vietnamese Restaurant,Accessories Store,Costume Shop
4,M6A,North York,Lawrence Manor,43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Gift Shop,Sporting Goods Shop,Miscellaneous Shop,Coffee Shop,Boutique,Vietnamese Restaurant,Accessories Store,Costume Shop


In [67]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = [] 
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [69]:
# 5. Examine Clusters

In [70]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


cluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


cluster5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [71]:

#show clusters

In [72]:
cluster1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0.0,Park,Food & Drink Shop,Construction & Landscaping,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
37,York,0.0,Park,Market,Fast Food Restaurant,Women's Store,Construction & Landscaping,Dog Run,Comic Shop,Farmers Market,Falafel Restaurant,Event Space
59,East York,0.0,Park,Convenience Store,Coffee Shop,Rental Car Location,Ethiopian Restaurant,Empanada Restaurant,Event Space,Electronics Store,Falafel Restaurant,Dessert Shop
69,North York,0.0,Park,Airport,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
70,North York,0.0,Park,Airport,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
88,North York,0.0,Park,Bakery,Construction & Landscaping,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
89,North York,0.0,Park,Bakery,Construction & Landscaping,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
90,North York,0.0,Park,Bakery,Construction & Landscaping,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
111,Central Toronto,0.0,Lawyer,Swim School,Bus Line,Park,Gourmet Shop,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
115,York,0.0,Park,Convenience Store,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


In [73]:
cluster2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1.0,Intersection,Financial or Legal Service,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant
2,Downtown Toronto,1.0,Coffee Shop,Bakery,Park,Pub,Restaurant,Café,Mexican Restaurant,Breakfast Spot,Ice Cream Shop,Bank
3,North York,1.0,Clothing Store,Furniture / Home Store,Gift Shop,Sporting Goods Shop,Miscellaneous Shop,Coffee Shop,Boutique,Vietnamese Restaurant,Accessories Store,Costume Shop
4,North York,1.0,Clothing Store,Furniture / Home Store,Gift Shop,Sporting Goods Shop,Miscellaneous Shop,Coffee Shop,Boutique,Vietnamese Restaurant,Accessories Store,Costume Shop
5,Queen's Park,1.0,Coffee Shop,Park,Gym,Sushi Restaurant,Diner,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Café
6,Downtown Toronto,1.0,Coffee Shop,Park,Gym,Sushi Restaurant,Diner,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Café
9,North York,1.0,Gym / Fitness Center,Japanese Restaurant,Caribbean Restaurant,Baseball Field,Café,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Dog Run
10,East York,1.0,Fast Food Restaurant,Pizza Place,Gym / Fitness Center,Intersection,Pharmacy,Café,Athletics & Sports,Bank,Gastropub,Women's Store
11,East York,1.0,Fast Food Restaurant,Pizza Place,Gym / Fitness Center,Intersection,Pharmacy,Café,Athletics & Sports,Bank,Gastropub,Women's Store
12,Downtown Toronto,1.0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Sporting Goods Shop,Italian Restaurant,Lingerie Store,Restaurant,Bookstore


In [74]:
cluster3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,Scarborough,2.0,Convenience Store,Playground,Women's Store,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
149,Central Toronto,2.0,Park,Playground,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
150,Central Toronto,2.0,Park,Playground,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
154,Scarborough,2.0,Park,Playground,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
155,Scarborough,2.0,Park,Playground,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
156,Scarborough,2.0,Park,Playground,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
157,Scarborough,2.0,Park,Playground,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
182,Downtown Toronto,2.0,Park,Playground,Trail,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Falafel Restaurant,Department Store


In [75]:
cluster4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,3.0,Fast Food Restaurant,Women's Store,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
8,Scarborough,3.0,Fast Food Restaurant,Women's Store,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
101,York,3.0,Fast Food Restaurant,Sandwich Place,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Festival
102,York,3.0,Fast Food Restaurant,Sandwich Place,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Festival
103,York,3.0,Fast Food Restaurant,Sandwich Place,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Festival
104,York,3.0,Fast Food Restaurant,Sandwich Place,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Festival


In [76]:
cluster5

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Scarborough,4.0,Bar,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Discount Store
21,Scarborough,4.0,Bar,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Discount Store
22,Scarborough,4.0,Bar,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Discount Store


In [77]:
print(cluster1.shape)
print(cluster2.shape)
print(cluster3.shape)
print(cluster4.shape)
print(cluster5.shape)

(20, 12)
(165, 12)
(8, 12)
(6, 12)
(3, 12)


**Cluster 2 is the biggest